# Andrew Lee DS210 HW3

## Part One

In [1]:
population = [1,2,3,4,5,6,7,8,9,10,11,12]

In [2]:
import numpy as np
import pandas as pd
import string

In [3]:
np.random.choice(population, size=10, replace=False)

array([ 9,  6,  7,  3, 10, 11,  1, 12,  8,  2])

In [4]:
cars = list(string.ascii_uppercase)[:18]

In [5]:
df1 = pd.DataFrame({'Name': np.random.choice(cars, size=10, replace=False),
                   'MPG': np.random.randint(5,50,10,int)})
df1

,Name,MPG
0,C,40
1,P,31
2,R,40
3,K,39
4,J,46
5,G,23
6,M,28
7,N,48
8,A,27
9,I,8


In [6]:
df2 = pd.DataFrame({'Name': np.random.choice(cars, size=10, replace=False),
                   'SecondsTo60': np.random.randint(1,20,10,int)})
df2

,Name,SecondsTo60
0,P,7
1,C,14
2,H,14
3,B,13
4,Q,8
5,E,4
6,L,5
7,N,14
8,I,16
9,O,6


In [7]:
df = df1.merge(df2, how='outer', on='Name')
df

,Name,MPG,SecondsTo60
0,C,40.0,14.0
1,P,31.0,7.0
2,R,40.0,NaN
3,K,39.0,NaN
4,J,46.0,NaN
5,G,23.0,NaN
6,M,28.0,NaN
7,N,48.0,14.0
8,A,27.0,NaN
9,I,8.0,16.0


In [8]:
df = df.fillna({'MPG': df.mean(axis=0)['MPG'], 'SecondsTo60':df.max(axis=0)['SecondsTo60']})
df

,Name,MPG,SecondsTo60
0,C,40.0,14.0
1,P,31.0,7.0
2,R,40.0,16.0
3,K,39.0,16.0
4,J,46.0,16.0
5,G,23.0,16.0
6,M,28.0,16.0
7,N,48.0,14.0
8,A,27.0,16.0
9,I,8.0,16.0


In [9]:
df['Name'] = df['Name'].apply(lambda name: 'Awesome ' + name)
df

,Name,MPG,SecondsTo60
0,Awesome C,40.0,14.0
1,Awesome P,31.0,7.0
2,Awesome R,40.0,16.0
3,Awesome K,39.0,16.0
4,Awesome J,46.0,16.0
5,Awesome G,23.0,16.0
6,Awesome M,28.0,16.0
7,Awesome N,48.0,14.0
8,Awesome A,27.0,16.0
9,Awesome I,8.0,16.0


In [10]:
df_eco = df[df['MPG'] > df.median(axis=0)['MPG']]
df_eco

,Name,MPG,SecondsTo60
0,Awesome C,40.0,14.0
2,Awesome R,40.0,16.0
3,Awesome K,39.0,16.0
4,Awesome J,46.0,16.0
7,Awesome N,48.0,14.0


## Part Two

In [11]:
from scipy.optimize import linprog

constraints = pd.read_csv('ingredients.csv', sep=',')
pastries = pd.read_csv('pastries.csv', sep=',')

In [12]:
constraints, pastries

(   Yeast  Flour  Sugar
 0   5.75  11.25   24.5,
               Name  Price  Yeast  Flour  Sugar
 0        Apple Pie   2.99   0.00   0.75   1.50
 1        Croissant   2.50   0.50   1.50   0.25
 2  Poppy seed roll   5.99   1.15   1.50   0.75)

In [13]:
supply = constraints.values[0]
supply 

array([ 5.75, 11.25, 24.5 ])

### Constraints

The ingredients dataframe is turned into an array for the optimization model. This array represents the target constraints for yeast, flower, and sugar respectively. 



In [14]:
recipes = pd.DataFrame(pastries, columns=['Yeast','Flour','Sugar']).values
recipes

array([[0.  , 0.75, 1.5 ],
       [0.5 , 1.5 , 0.25],
       [1.15, 1.5 , 0.75]])

In [15]:
inequalities = np.transpose(recipes)
inequalities

array([[0.  , 0.5 , 1.15],
       [0.75, 1.5 , 1.5 ],
       [1.5 , 0.25, 0.75]])

### Inequalities
The recipes array is created from the yeast, flower, and sugar columns. These numbers represent the required quantity of each ingredient for the specified product. The columns represent a specific ingredient and this is an issue for linprog because it will read the given inequality horizontally not vertically. Ideally we want an array which represents the set of inequalities:


Given A = # of apple pies, C = # of croissants, and P = # of poppy seed rolls
```
0.00A + 0.50C + 1.15P <= 5.75
0.75A + 1.50C + 1.50P <= 11.25
1.50A + 0.25C + 0.75P <= 24.5
```
The corresponding array:
```python
[0.00, 0.50, 1.15]
[0.75, 1.50, 1.50]
[1.50, 0.25, 0.75]
```
with corresponding constraints:
```python
[ 5.75, 11.25, 24.5]
```
-- -- --
When creating array from columns of the dataframe, the array is flipped, so to correct it and formulate it into the desired array, we transpose it using `np.transpose(recipes)`

In [22]:
obj = [-2.99, -2.50, -5.99]
bnd = [(0, int(40)),(0, int(40)),(0, int(40))]

### Objective Coefficients

The obj list represents the coefficients of the objective function. This is the function we ultimately want to optimize. These coeffiecnts are the prices of each item. The important part to realize is that this optimization function has to be converted to minimization instead of maximization because `linprog` only does minimization. To account for this, we make the coefficients negative.

### Bounds

This array will set the bounds of each variable, in this case, A, C, and P. We specify here that we dont want these variables to be negative nor do we want them to be more than 50. This number is arbitrary and can technically be positive infinity. We really just want to make sure these variables can be maximized in order to find the maximum revenue.

In [25]:
opt = linprog(c=obj, A_ub=inequalities, b_ub=supply,
            bounds=bnd, method="revised simplex")

In [26]:
opt

     con: array([], dtype=float64)
     fun: -44.900000000000006
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([ 0.  ,  0.  , 13.25])
  status: 0
 success: True
       x: array([5., 0., 5.])

### Result

According to the linear program optimization model, the best combination of pastries to maximize sales revenue is **5 Apple Pies and 5 Poppy Seed Rolls**. This is the best combination given our ingredient constraints. The max revenue with this combination is **$44.90** .